<a href="https://www.kaggle.com/code/pietrocaforio/unimodal-ct-training-kaggle?scriptVersionId=197018220" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Train unimodal CT

In [1]:
!git clone https://github.com/PietroCaforio/research-biocv-proj
!cd research-biocv-proj && git switch dev

Cloning into 'research-biocv-proj'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 181 (delta 97), reused 106 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (181/181), 3.43 MiB | 19.64 MiB/s, done.
Resolving deltas: 100% (97/97), done.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [2]:
!cd research-biocv-proj && git pull

Already up to date.


In [3]:
!pip install wandb

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")

In [5]:
import wandb
wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
import sys
from pathlib import Path

# Add the 'data' directory to sys.path
sys.path.append(str(Path('research-biocv-proj').resolve()))
from data.unimodal import *
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader

from sklearn.utils.class_weight import compute_class_weight

### Train ResNet model

In [7]:
def train(model,config, run_name=None):
  wandb.init(
    # set the wandb project where this run will be logged
    project="unimodal_ct_training",
    name = run_name,
    # track hyperparameters and run metadata
    config=config
  )
  optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])
  criterion = nn.CrossEntropyLoss(weight = torch.tensor(config["class_weights"], dtype=torch.float).to(device) )
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = config["reduce_lr_factor"], patience = config["patience"])
  # Training loop
  num_epochs = config["epochs"]
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0

      for batch in train_loader:
          frames = batch['frame'].float().to(device)
          labels = batch['label'].long().to(device)

          optimizer.zero_grad()
          outputs = model(frames)
          loss = criterion(outputs.logits, labels)

          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

      # Validation loop
      model.eval()
      val_loss = 0.0
      correct = 0
      total = 0
      # Initialize counters for each class (G1, G2, G3)
      correct_per_class = [0, 0, 0]  # For G1, G2, G3
      total_per_class = [0, 0, 0]  # For G1, G2, G3

      with torch.no_grad():
          for batch in val_loader:
              frames = batch['frame'].float().to(device)
              labels = batch['label'].long().to(device)

              outputs = model(frames)
              loss = criterion(outputs.logits, labels)

              val_loss += loss.item()
              _, predicted = torch.max(outputs.logits, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

              # Calculate accuracy per class
              for i in range(3):  # We have 3 classes: G1 (0), G2 (1), G3 (2)
                  correct_per_class[i] += ((predicted == i) & (labels == i)).sum().item()
                  total_per_class[i] += (labels == i).sum().item()
      scheduler.step(val_loss)
      # Compute total accuracy and per-class accuracy
      total_accuracy = 100 * correct / total
      class_accuracy = [(100 * correct_per_class[i] / total_per_class[i]) if total_per_class[i] > 0 else 0 for i in range(3)]
      print(f"Validation Loss: {val_loss/len(val_loader)}, Total Accuracy: {total_accuracy:.2f}%")
      print(f"Accuracy per class - G1: {class_accuracy[0]:.2f}%, G2: {class_accuracy[1]:.2f}%, G3: {class_accuracy[2]:.2f}%")
      # log metrics to wandb
      wandb.log({"Total Accuracy": total_accuracy, "Validation Loss": val_loss/len(val_loader), "G1_Acc":class_accuracy[0], "G2_Acc":class_accuracy[1], "G3_Acc":class_accuracy[2]})
  wandb.finish()  

In [8]:
import torch.nn as nn
import torch.optim as optim
from transformers import ResNetForImageClassification

In [9]:
train_dataset = UnimodalCTDataset(split='train',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/" )
val_dataset = UnimodalCTDataset(split='val',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [10]:
print(f"Training set stats:{train_dataset.stats()}")
print(f"Validation set stats:{val_dataset.stats()}")

Training set stats:{'length': 2292, 'class_frequency': {'G1': 78, 'G2': 1543, 'G3': 671}}
Validation set stats:{'length': 335, 'class_frequency': {'G1': 37, 'G2': 166, 'G3': 132}}


In [11]:
labels = []
for sample in train_dataset:
    labels.append(sample["label"])
labels = np.array(labels)
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)

In [12]:
print(class_weights)

[9.79487179 0.49513934 1.13859911]


### Resnet-50

In [13]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-50')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [15]:
config={
    "learning_rate": 1e-4,
    "architecture": "microsoft/resnet-50 new1",
    "epochs": 100,
    "weight_decay": 1e-4,
    "reduce_lr_factor": 0.2,
    "patience": 10,
    "class_weights": class_weights
    }
train(model, config, run_name = config["architecture"])

wandb: Currently logged in as: pietro-caforio (pietro-caforio-politecnico-di-milano). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240917_130832-f7iold93
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-50 new1
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/f7iold93


Epoch 1, Loss: 0.9871006260315577
Validation Loss: 0.9886575910178098, Total Accuracy: 57.31%
Accuracy per class - G1: 0.00%, G2: 83.73%, G3: 40.15%
Epoch 2, Loss: 0.5770902203189002
Validation Loss: 0.880466715856032, Total Accuracy: 62.09%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 62.88%
Epoch 3, Loss: 0.1827401276677847
Validation Loss: 4.919572115621784, Total Accuracy: 60.90%
Accuracy per class - G1: 0.00%, G2: 75.90%, G3: 59.09%
Epoch 4, Loss: 0.05017987359315157
Validation Loss: 8.461934896016663, Total Accuracy: 59.70%
Accuracy per class - G1: 0.00%, G2: 75.90%, G3: 56.06%
Epoch 5, Loss: 0.02329029268326445
Validation Loss: 11.819217964498835, Total Accuracy: 51.64%
Accuracy per class - G1: 0.00%, G2: 77.11%, G3: 34.09%
Epoch 6, Loss: 0.012230305770774268
Validation Loss: 14.994751256467266, Total Accuracy: 58.51%
Accuracy per class - G1: 0.00%, G2: 76.51%, G3: 52.27%
Epoch 7, Loss: 0.007684720259728945
Validation Loss: 16.390077114697885, Total Accuracy: 55.82%
Accuracy 

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          G2_Acc █▁▂▃▃▂▂▃▂▂▃▂▂▃▂▂▃▂▂▃▂▃▃▂▃▂▃▃▃▃▃▃▃▂▂▃▂▂▂▃
wandb:          G3_Acc ▄█▇▅▁▃▃▁▁▅▂▃▃▃▃▃▃▃▄▃▃▃▂▃▂▃▃▃▃▂▂▃▂▃▄▃▃▃▃▃
wandb:  Total Accuracy ▆█▇▅▁▃▃▁▁▅▂▃▃▃▃▃▃▃▄▃▃▃▂▃▂▃▃▃▃▂▂▃▂▃▄▃▃▃▃▃
wandb: Validation Loss ▁▂▄▅▆▇▇██▆▇▇▇▇███▇▇███▇▇██▇▇███▇██████▇▇
wandb: 
wandb: Run summary:
wandb:          G1_Acc 0.0
wandb:          G2_Acc 77.71084
wandb:          G3_Acc 33.33333
wandb:  Total Accuracy 51.64179
wandb: Validation Loss 29.74184
wandb: 
wandb: 🚀 View run microsoft/resnet-50 new1 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/f7iold93
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240917_130832-f7

### Resnet-18

In [16]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-18')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [18]:
config={
    "learning_rate": 1e-3,
    "architecture": "microsoft/resnet-18 new1",
    "epochs": 100,
    "weight_decay": 1e-6,
    "reduce_lr_factor": 0.25,
    "patience": 10,
    "class_weights": class_weights
    }
train(model, config, run_name = config["architecture"])

wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240917_134729-p19cq7bs
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-18 new1
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/p19cq7bs


Epoch 1, Loss: 0.19337432938563223
Validation Loss: 3.3165150579403773, Total Accuracy: 66.57%
Accuracy per class - G1: 0.00%, G2: 90.36%, G3: 55.30%
Epoch 2, Loss: 0.0627752602401112
Validation Loss: 1.6806126961230554, Total Accuracy: 65.37%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 71.21%
Epoch 3, Loss: 0.015546958541462017
Validation Loss: 2.951860401058432, Total Accuracy: 57.31%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 50.76%
Epoch 4, Loss: 0.007152476225908483
Validation Loss: 1.4514926813479343, Total Accuracy: 70.45%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 84.09%
Epoch 5, Loss: 0.0006888409294939871
Validation Loss: 1.8640455612489446, Total Accuracy: 65.07%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 70.45%
Epoch 6, Loss: 0.00023289929249080564
Validation Loss: 1.832097603854808, Total Accuracy: 62.09%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 62.88%
Epoch 7, Loss: 0.00012406714293117856
Validation Loss: 1.8743066547269172, Total Accuracy: 63.58

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          G2_Acc █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          G3_Acc ▃▁▅▆█▆▆▅▆█▆▅▆▅▅▅▅▆▅▆▅▅▅▅▆▅▅▆▅▅▅▅▅▅▅▅▅▆▅▆
wandb:  Total Accuracy █▁▅▆▇▆▆▅▅▇▅▅▅▅▅▅▅▆▅▅▅▅▄▅▆▅▅▆▅▅▅▅▅▅▅▄▅▅▅▆
wandb: Validation Loss █▆▁▁▃▂▃▂▃▄▃▃▃▃▃▃▂▃▂▃▃▂▂▃▃▃▃▃▃▃▃▃▃▂▃▃▃▃▃▃
wandb: 
wandb: Run summary:
wandb:          G1_Acc 0.0
wandb:          G2_Acc 75.3012
wandb:          G3_Acc 65.90909
wandb:  Total Accuracy 63.28358
wandb: Validation Loss 2.26276
wandb: 
wandb: 🚀 View run microsoft/resnet-18 new1 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/p19cq7bs
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240917_134729-p19c

### Resnet-34

In [19]:

model = ResNetForImageClassification.from_pretrained('microsoft/resnet-34')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [21]:
config={
    "learning_rate": 1e-3,
    "architecture": "microsoft/resnet-34 new1",
    "epochs": 100,
    "weight_decay": 1e-5,
    "reduce_lr_factor": 0.5,
    "patience": 20,
    "class_weights": class_weights
    }
train(model, config, run_name = config["architecture"])


wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240917_141250-881pup6m
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-34 new1
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/881pup6m


Epoch 1, Loss: 0.389800224504951
Validation Loss: 2.6990506747229532, Total Accuracy: 68.66%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 79.55%
Epoch 2, Loss: 0.20478206949903527
Validation Loss: 9.331513213840397, Total Accuracy: 53.13%
Accuracy per class - G1: 0.00%, G2: 90.36%, G3: 21.21%
Epoch 3, Loss: 0.03180657046525286
Validation Loss: 2.4699919758650966, Total Accuracy: 61.19%
Accuracy per class - G1: 0.00%, G2: 87.95%, G3: 44.70%
Epoch 4, Loss: 0.0020300203656612816
Validation Loss: 3.2994927382662187, Total Accuracy: 59.40%
Accuracy per class - G1: 0.00%, G2: 82.53%, G3: 46.97%
Epoch 5, Loss: 0.0011189640523904625
Validation Loss: 3.3597943088878477, Total Accuracy: 73.73%
Accuracy per class - G1: 0.00%, G2: 83.73%, G3: 81.82%
Epoch 6, Loss: 0.053633750438368426
Validation Loss: 1.9088656275776956, Total Accuracy: 74.93%
Accuracy per class - G1: 0.00%, G2: 95.18%, G3: 70.45%
Epoch 7, Loss: 0.04681207738677687
Validation Loss: 3.4559623643925246, Total Accuracy: 56.72%
Acc

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          G2_Acc ▁▅█▆▆▆▆▆▆▆▆▆▆▆▆▆▆▄▁▁▁▁▁▁▁▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
wandb:          G3_Acc █▁▆▁█▇▆▆▄▁▆▄▃▄▃▃▁▂▂▂▃▂▂▂▂▃▂▂▂▂▂▃▂▂▂▂▃▃▃▃
wandb:  Total Accuracy ▆▃█▃█▇▇▇▅▃▇▆▄▅▅▅▃▃▁▁▁▁▁▁▁▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
wandb: Validation Loss ▂▂▁▄▅▅▆▆▆▆▆▆▆▆▆▆▆▅▄▆▆▆▆▇█▄▄▄▅▄▅▇█▇▇▅▅▅▆▆
wandb: 
wandb: Run summary:
wandb:          G1_Acc 0.0
wandb:          G2_Acc 89.75904
wandb:          G3_Acc 51.51515
wandb:  Total Accuracy 64.77612
wandb: Validation Loss 5.38336
wandb: 
wandb: 🚀 View run microsoft/resnet-34 new1 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/881pup6m
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240917_141250-881